In [310]:
import pandas as pd
import numpy as np
import pickle

In [315]:
def group_data(g_df):
    cols = ['bac', 'iac', 'aac', 'nac', 'wac']

#     # These should be the same across the group
#     out = g_df[base_cols].iloc[0]

    tot = g_df[(g_df['Sex.id'] == 'totsex') &
         (g_df['Hisp.id'] == 'tothisp')].iloc[0][cols]

    tot = tot.rename({x: 't_'+x for x in cols})

    male = g_df[(g_df['Sex.id'] == 'male') &
         (g_df['Hisp.id'] == 'tothisp')].iloc[0][cols]
    male = male.rename({x: 'm_'+x for x in cols})

    female = g_df[(g_df['Sex.id'] == 'female') &
         (g_df['Hisp.id'] == 'tothisp')].iloc[0][cols]
    female = female.rename({x: 'f_'+x for x in cols})

#     out = pd.concat([out, male, female])
    out = pd.concat([male, female])


    out['t_hisp'] = g_df[(g_df['Hisp.id'] == 'hisp') &
         (g_df['Sex.id'] == 'totsex')][cols].iloc[0].astype(int).sum()
    out['m_hisp'] = g_df[(g_df['Hisp.id'] == 'hisp') &
         (g_df['Sex.id'] == 'male')][cols].iloc[0].astype(int).sum()
    out['f_hisp'] = g_df[(g_df['Hisp.id'] == 'hisp') &
         (g_df['Sex.id'] == 'female')][cols].iloc[0].astype(int).sum()

    return out

In [316]:
def process_census_data(file_name):

    df = pd.read_csv(file_name, encoding='latin')
    # Drops the header, because it has two for some reason. Dropped becase Rochita fixed this
#     df=df[1:]

    df['fips'] = df['GEO.id2'].astype(int)
    df['year'] = df['Year.id'].str.slice(start=4).astype(int)
    
    df['county_name'] = df['GEO.display-label']
    df['state'] = df['county_name'].str.split(',').apply(lambda x: x[1].strip())
    df['county_name'] = df['county_name'].str.split(',').apply(lambda x: x[0].strip())

    df = df[(df.year == 2012) | 
            (df.year == 2015)]

    groups = df.groupby(['year', 'fips', 'county_name', 'state'])   

    f_df = groups.apply(group_data).reset_index()

    return f_df

In [317]:
file_name = 'county_demographics.csv'

In [318]:
df = process_census_data(file_name)

In [321]:
df_det = pd.read_csv('county_facts.csv')

In [322]:
new_names = {'fips': 'fips',
             'AGE135214': 'age < 5',
             'AGE295214': 'age < 18',
             'AGE775214': 'age > 65',
             'POP715213': 'same house > 1 yr', 
             'POP645213': 'foreign_born',
             'POP815213': 'non_english',
             'EDU635213': '>= high school',
             'EDU685213': '>= bachelor',
             'VET605213': 'veterans',
             'LFE305213': 'avg commute',
             'HSG010214': 'housing units',
             'HSG445213': 'homeownership rate', 
             'HSG096213': 'multiunit housing',
             'HSG495213': 'home value',
             'HSD410213': 'households',
             'HSD310213': 'peeps per house', 
             'INC910213': 'income per cap',
             'INC110213': 'household income',
             'PVY020213': 'below poverty',
             'BZA010213': 'nonfarm homes', 
             'BZA110213': 'nonfarm empl',
             'BZA115213': 'nonfarm empl % change', 
             'NES010213': 'nonemployer establishments', 
             'SBO001207': 'total firms',
             'SBO315207': 'black firms',
             'SBO115207': 'native american firms',
             'SBO215207': 'asian firms', 
             'SBO515207': 'hawaiian firms',
             'SBO415207': 'hispanic firms',
             'SBO015207': 'women firms',
             'MAN450207': 'manufacturing shipments',
             'WTN220207': 'merchant wholesaler sales',
             'RTN130207': 'retail sales',
             'RTN131207': 'retail sales per capita',
             'AFN120207': 'service sector sales',
             'BPS030214': 'building permits',
             'LND110210': 'land area',
             'POP060210': 'pop density'}

In [337]:
list(new_names.values())

['black firms',
 'land area',
 '>= bachelor',
 'hawaiian firms',
 'total firms',
 'native american firms',
 'pop density',
 'non_english',
 'same house > 1 yr',
 'veterans',
 'multiunit housing',
 'asian firms',
 'fips',
 'home value',
 'nonfarm empl % change',
 'homeownership rate',
 'hispanic firms',
 'nonfarm homes',
 '>= high school',
 'retail sales',
 'service sector sales',
 'age > 65',
 'merchant wholesaler sales',
 'avg commute',
 'nonemployer establishments',
 'retail sales per capita',
 'foreign_born',
 'peeps per house',
 'households',
 'age < 18',
 'building permits',
 'nonfarm empl',
 'manufacturing shipments',
 'income per cap',
 'below poverty',
 'age < 5',
 'housing units',
 'women firms',
 'household income']

In [323]:
df_det = df_det[list(new_names.keys())].rename(columns=new_names)

In [324]:
# df_det_dict = pd.read_csv('county_facts_dictionary.csv')

In [325]:
df = df.merge(df_det, how='left', on='fips')

In [326]:
df = df[df.state != 'Alaska']

In [327]:
df.loc[df.year == 2015, 'year'] = 2016

In [328]:
df_voting = pd.read_csv('US_County_Level_Presidential_Results_12-16.csv')

In [329]:
df_voting = df_voting[df_voting.state_abbr != 'AK']

In [330]:
# This county got renamed
df_voting.loc[df_voting.FIPS == 46113, 'FIPS'] = 46102
# This county is only accessible by mule trail. Thanks wikipedia
df = df[df.fips != 15005]

In [331]:
cols_12 = {'FIPS': 'fips', 
           'votes_dem_2012': 'votes_dem',
           'votes_gop_2012': 'votes_rep',
           'total_votes_2012': 'total_votes'
}
df_12 = df_voting.loc[pd.notnull(df_voting.total_votes_2012), list(cols_12.keys())].rename(columns=cols_12)
df_12['year'] = 2012

cols_16 = {'FIPS': 'fips', 
           'votes_dem_2016': 'votes_dem',
           'votes_gop_2016': 'votes_rep',
           'total_votes_2016': 'total_votes'

}
df_16 = df_voting.loc[pd.notnull(df_voting.total_votes_2016), list(cols_16.keys())].rename(columns=cols_16)
df_16['year'] = 2016


In [332]:
df_voting = pd.concat([df_12, df_16])

In [333]:
df = df.merge(df_voting, on=['fips', 'year'], how='left')

In [334]:
df.to_csv('processed_data.csv', index=False)